# Introduction to LangChain

- https://console.mistral.ai/
- https://www.langchain.com/



In [1]:
# Get the API key here and add it to the secrets (left).
from google.colab import userdata
api_key = userdata.get("mistralapikey")

SecretNotFoundError: Secret mistralapikey does not exist.

In [ ]:
!pip install -U langchain-core langchain-mistralai langchain-community langchain-chroma

# Imports.


In [ ]:
import json
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_mistralai.chat_models import ChatMistralAI

from langchain_core.globals import set_verbose, set_debug
set_verbose(False)
set_debug(False)

import logging
logging.getLogger().setLevel(logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

## Getting started.

- https://docs.mistral.ai/getting-started/models/

In [ ]:
llm = ChatMistralAI(
    api_key=api_key,
    model="mistral-medium-latest"
)

messages = [
    SystemMessage(
        content="You are a friendly AI assistant that speaks English but sometimes uses German words."
    ),
    HumanMessage(
        content="Write a poem about love."
    )
]
result = llm.invoke(messages)
print(result.content)

In [ ]:
print(json.dumps(result.response_metadata, indent=4))

## Use streaming.

In [ ]:
async for chunk in llm.astream(messages):
    print(chunk.content, end="", flush=True)

## Translation

In [ ]:
messages = [
    SystemMessage(
        content=""
            "You are a friendly AI assistant."
            " Your specialty are great translations. Answer with the translation first. And then explain it in detail. Explanation as a bulleted list please. Use HTML tags."
    ),
    HumanMessage(
        content=""
            "L'homme est libre au moment qu'il veut l'être."
        )
]
result = llm.invoke(messages)
result.content

In [ ]:
print(result.content)

## Chain example: Parsing.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = llm.invoke(messages)
print(result)
parsed_result = parser.invoke(result)
print(parsed_result)

In [ ]:
chain = llm | parser
chain.invoke(messages)

## Summarization.

https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/

In [ ]:
# Get a file.
!wget https://raw.githubusercontent.com/vilmibm/lovecraftcorpus/master/ulthar.txt

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("ulthar.txt")
documents = loader.load()

first_document_content = documents[0].page_content

summary_prompt = f"Please summarize the following document:\n\n{first_document_content}"
messages = [
    SystemMessage(
        content="You are a friendly AI assistant that speaks English."
                "You write really good summaries."
                "You sometimes use bulleted lists but not all the time."
    ),
    HumanMessage(content=summary_prompt)
]

chain = llm | parser
summary = chain.invoke(messages)
summary

## Advanced summarization.

In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.invoke(documents)["output_text"]
summary

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")
summary = chain.invoke(documents)["output_text"]
summary

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")
summary = chain.invoke(documents)["output_text"]
summary

## Structured output.

In [ ]:
import json
from langchain_core.output_parsers import JsonOutputParser

messages = [
    SystemMessage(
        content="You are a friendly AI assistant that speaks English."
                "Your specialty is extracting structured output in JSON."
    ),
    HumanMessage(
        content=f"List all the characters and what you know about them as JSON:\n\n{first_document_content}"
    )
]

chain = llm | JsonOutputParser()
structured_output = chain.invoke(messages)
print(json.dumps(structured_output, indent=4))

## Advanced structured output with Pydantic

- https://docs.pydantic.dev/latest/

In [ ]:
from typing import List
from pydantic import BaseModel
from langchain.chains import create_extraction_chain_pydantic


class Person(BaseModel):
    first_name: str
    last_name: str
    known_facts: str

class PersonGroup(BaseModel):
    persons: List[Person]

llm_small = ChatMistralAI(
    api_key=api_key,
    model="mistral-small-latest" # Medium does not have function calling.
)

chain = llm_small.with_structured_output(PersonGroup)
structured_output = chain.invoke(summary_prompt)
print(structured_output)
print()
print(structured_output.model_dump_json(indent=4))

## Classification.

In [ ]:
from pydantic import BaseModel, Field


class Classification(BaseModel):
    sentiment: str = Field(
        ...,
        description="describes the sentiment of the statement",
        enum=["negative", "neutral", "positive"]
    )
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[0, 1, 2, 3],
    )
    language: str = Field(
        ...,
        description="describes the language of the statement",
        enum=["english", "french", "german", "other"]
    )

statements = [
    "I absolutely love this new restaurant! The food is amazing, and the service is top-notch.",
    "Le service client ici est terrible, et je ne reviendrai jamais.",
    "Ich bin gleichgültig gegenüber der neuen Politik; sie betrifft mich nicht wirklich.",
    "Your recent actions were completely unacceptable, and they have consequences.",
    "Quel beau jour ! Je me sens si heureux et en paix.",
    "Die Art und Weise, wie Sie die Situation gehandhabt haben, war sehr enttäuschend und unprofessionell.",
    "Creo que la presentación estuvo bien, pero podría mejorar.",
    "You have no right to speak to me that way! It's utterly disrespectful.",
    "Ce livre est très intéressant, et j'ai beaucoup aimé le lire.",
    "Ihre Bemühungen bei dem Projekt waren bestenfalls mittelmäßig, und wir müssen das besprechen.",
    "Hab SoSlI' Quch!"
]

for statement in statements:
    chain = llm_small.with_structured_output(Classification)
    structured_output = chain.invoke(statement)
    print(statement)
    print(structured_output.dict())
    print("")

## Tool use.

In [ ]:
from langchain_core.tools import tool

@tool
def sum_tool(numbers:list) -> int:
    """Sum up numbers."""
    return sum(numbers)

print(sum_tool.name)
print(sum_tool.description)
print(sum_tool.args)

numbers = [42, 308423, 666, 1000000, 1729, -1245, 768]
numbers_string = ", ".join(str(n) for n in numbers)

# Sanity.
print("Expected:", sum(numbers))
print("")

# Create the prompts.
system_prompt = f"You are a friendly AI assistant that speaks English. You are good at math."
sum_prompt = f"Please sum up the following numbers: {numbers_string}."
messages = [
    SystemMessage(content=sum_prompt),
    HumanMessage(content=numbers_string)
]

# Without tools.
print("Without tools:")
print(llm_small.invoke(messages))
print("")

# With tools.
print("With tools:")
llm_small_with_tools = llm_small.bind_tools([sum_tool])
print(llm_small_with_tools.invoke(messages))
chain = llm_small_with_tools | (lambda x: x.tool_calls[0]["args"]) | sum_tool
print(chain.invoke(messages))

## Loading PDFs.


In [ ]:
!wget https://www.pileface.com/sollers/pdf/Zarathustra.pdf
!pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Zarathustra.pdf")
pages = loader.load_and_split()

for page in pages[:2]:
    print(page.page_content)
    print("")

## Web Loader

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://de.wikipedia.org/wiki/Heilbronn")

pages = loader.load_and_split()

for page in pages[:2]:
    print(page.page_content)
    print("")

## Gradio chat.

https://www.gradio.app/

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()

## Vector Databases.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings

embeddings_model = MistralAIEmbeddings(
    api_key=api_key,
    model="mistral-embed"
)

embedding = embeddings_model.embed_query("This is a test, I want to embed.")
print(len(embedding))

In [ ]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator("embedding_distance", embeddings=embeddings_model)

distance = evaluator.evaluate_strings(
    prediction="Dune is a great movie.",
    reference="I like the Star Wars series."
)
print(distance)

distance = evaluator.evaluate_strings(
    prediction="Dune is a great movie.",
    reference="Hi. I am Tristan. I love teaching AI."
)
print(distance)

In [ ]:
!wget https://raw.githubusercontent.com/vilmibm/lovecraftcorpus/master/mountains_of_madness.txt

## Let us use Chroma.

- https://www.trychroma.com/

In [ ]:
from langchain_chroma import Chroma

raw_documents = TextLoader("mountains_of_madness.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
print(f"Got {len(documents)} documents after splitting")

print(documents[0])

Fill the database.

In [ ]:
database = Chroma.from_documents(documents, embeddings_model)

Query the database.

In [ ]:
query = "What is an Old One?"
docs = database.similarity_search(query)
docs[0].page_content

In [ ]:
query = "What is an Old One?"
docs = database.similarity_search_with_score(query)
docs[0][0].page_content, docs[0][1]

## Talk to document.

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

qa_chain = load_qa_chain(llm)

template="""Given the following conversation history and a new user question, generate a standalone question.
Conversation history:
{chat_history}
New question: {question}
Standalone question:"""

question_generator_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=template
)

question_generator_chain = LLMChain(
    llm=llm,
    prompt=question_generator_prompt
)

retrieval_chain = ConversationalRetrievalChain(
    retriever=database.as_retriever(search_kwargs={"k": 5}),
    combine_docs_chain=qa_chain,
    question_generator=question_generator_chain
)

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    history_langchain_format.append(HumanMessage(content=message))

    response = retrieval_chain(
        {"question": message, "chat_history": history_langchain_format}
    )

    return response["answer"]

gr.ChatInterface(predict).launch()

# Software development

In [ ]:
messages = [
    SystemMessage(
        content="You are a 150K EUR/year principal software engineer. You write the best code in the world."
    ),
    HumanMessage(
        content="Implement Conway's game of life in Python."
    )
]
result = llm.invoke(messages)
print(result.content)

In [ ]:
code = """
class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task: str):
        if not isinstance(task, str) or not task.strip():
            raise ValueError("Task must be a non-empty string")
        self.tasks.append({"task": task, "completed": False})

    def remove_task(self, task: str):
        for t in self.tasks:
            if t["task"] == task:
                self.tasks.remove(t)
                return
        raise ValueError("Task not found")

    def mark_completed(self, task: str):
        for t in self.tasks:
            if t["task"] == task:
                t["completed"] = True
                return
        raise ValueError("Task not found")

    def get_tasks(self, completed=None):
        if completed is None:
            return self.tasks
        return [t for t in self.tasks if t["completed"] == completed]

    def clear_completed(self):
        self.tasks = [t for t in self.tasks if not t["completed"]]

"""

messages = [
    SystemMessage(
        content="You are a 150K EUR/year principal software engineer. You write the best code in the world."
    ),
    HumanMessage(
        content=f"Here is some code:\n\n'''\n{code}\n'''\n\nPlease write unit tests."
    )
]
result = llm.invoke(messages)
print(result.content)

In [ ]:
messages = [
    SystemMessage(
        content="You are a 150K EUR/year principal code reviewer. You write code reviews even god has never seen."
    ),
    HumanMessage(
        content=f"Here is some code:\n\n'''\n{code}\n'''\n\nPlease a code review.."
    )
]
result = llm.invoke(messages)
print(result.content)

TODO: https://blog.langchain.dev/reflection-agents/